### Flavors of Boosting

In this notebook, we build a photometric redshift estimator using various boosting methods: AdaBoost and various flavors of Gradient Boosted Trees (GBM, HistGBM, and XGBoost). We also look at using RandomizedSearchCV in order to improve our exploration of parameter space.

Our goal is to estimate photometric redshifts starting from observations of galaxy magnitudes in six different photometric bands (u, g, r, i, z, y). 

It accompanies Chapter 6 of the book (3 of 4).

Copyright: Viviana Acquaviva (2023); see also other data credits below.

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

Essentially, we try to reproduce/improve upon the results of [this paper](https://arxiv.org/abs/1903.08174), for which the data are public and available [here](http://d-scholarship.pitt.edu/36064/).

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
#matplotlib.rcParams.update({'figure.autolayout': True})
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, KFold, cross_val_predict, GridSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor

### We can read the data set with the selections applied in the previous notebook.

In [ ]:
sel_features = pd.read_csv('sel_features.csv', sep = '\t')

In [ ]:
sel_target = pd.read_csv('sel_target.csv')

In [ ]:
sel_features.shape

(6307, 6)

In [ ]:
sel_target.values.ravel() #changes shape to 1d row-like array

array([1.0034, 0.9023, 0.4242, ..., 0.3703, 0.371 , 0.7333])

#### In the notebook "BoostingDecisions", we showed that for AdaBoost, stacking learners that are too weak doesn't help.

This allows us to run a more informed parameter optimization.

In [ ]:
# %%time Wall time on my machine was 1 min 17 s

# Note: the variables after the "time" magic command are not updated - 
# e.g. the "model" object will not be the one defined later in the cell.
# This may depend on the Jupyter notebook version.

parameters = {'base_estimator__max_depth':[6,10,None], 'loss':['linear','square'], 'n_estimators':[20,50,100], 'learning_rate': [0.3,0.5,1.0]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(AdaBoostRegressor(base_estimator=DecisionTreeRegressor()), parameters, \
                     cv = KFold(n_splits=5, shuffle=True, random_state = 5), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best params, best score: 0.7581 {'base_estimator__max_depth': None, 'learning_rate': 0.3, 'loss': 'square', 'n_estimators': 100}


We can take a look at the winning model scores; in this case, we also pay attention to the standard deviation of test scores, because we want to know what differences are statistically significant when we compare different models.

In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

,params,mean_test_score,std_test_score,mean_train_score
41,"{'base_estimator__max_depth': None, 'learning_rate': 0.3, 'loss': 'square', 'n_estimators': 100}",0.758134,0.032466,0.999978
47,"{'base_estimator__max_depth': None, 'learning_rate': 0.5, 'loss': 'square', 'n_estimators': 100}",0.750198,0.033006,0.999732
49,"{'base_estimator__max_depth': None, 'learning_rate': 1.0, 'loss': 'linear', 'n_estimators': 50}",0.746901,0.055565,0.999746
37,"{'base_estimator__max_depth': None, 'learning_rate': 0.3, 'loss': 'linear', 'n_estimators': 50}",0.745610,0.035138,0.999992
52,"{'base_estimator__max_depth': None, 'learning_rate': 1.0, 'loss': 'square', 'n_estimators': 50}",0.743821,0.038081,0.986022


We can see that the standard deviation is 0.03 - giving us a hint of what's significant - and that a few different models have similar scores. If you change the random seed in the cross validation, the scores will change by a similar amount, and the best model may change as well.

Additionally, the resulting scores will not be exactly reproducible because there is another random component in the adaptive learning set (this means that if you run the cross_validate function using the best model from above, you might get a different average score!)

#### Let's pick the best model and check the scores. We should do nested cross validation to get the generalization errors right - but if we are just comparing models, this is ok.

In [ ]:
bm = model.best_estimator_

Let's generate predicted values.

In [ ]:
y_pred_bm = cross_val_predict(bm, sel_features, sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle = True, random_state=10))

Calculate outlier fraction and NMAD:

In [ ]:
print(np.round(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()),3))

print(np.round(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())),3))

0.045
0.032


These are actually better than what we obtained for the Random Forests model! But is the difference statistically significant? One way to explore this is by generating several sets of predictions, and calculating their standard deviation.

In [ ]:
seeds = np.random.choice(100,8, replace = False) #pick 8 different seeds

olf = np.zeros(8)
NMAD = np.zeros(8)

for i in range(8): #A bit rough, but it gives a sense of what happens by varying the random seeds!
    print('Iteration', i) #this is just to see the progress.
    ypred = cross_val_predict(bm, sel_features, sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle=True, random_state=seeds[i]))
    olf[i] = len(np.where(np.abs(sel_target.values.ravel()-ypred)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel())
    NMAD[i] = 1.48*np.median(np.abs(sel_target.values.ravel()-ypred)/(1 + sel_target.values.ravel()))

print('OLF avg/std:, {0:.5f}, {1:0.5f}'.format(olf.mean(), olf.std()))
print('NMAD avg/std:, {0:.5f}, {1:0.5f}'.format(NMAD.mean(), NMAD.std()))

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
OLF avg/std:, 0.04533, 0.00104
NMAD avg/std:, 0.03223, 0.00023


The result seems to be relatively solid, indicating that Boosting methods might be slightly better than RF when we take into account not just the R2 score, but the specific metrics we are monitoring for this problem.

### The next step is to compare Adaptive Boosting with different Gradient Boosted Trees algorithms. 

We begin by using sklearn's GBM, then we move on to the lighter version, HistGBM, and finally we consider one of the most popular GBT-based algorithm, XGBoost.

We also take a look at the possibility of using a Randomized Search instead of a Grid Search in order to speed up our optimization process.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

The parameters depend on the particular implementation.

In the sklearn formulation, the parameters of each tree are essentially the same we have for Random Forests; additionally we have the "learning_rate" parameter, which dictates how much each tree contribute to the final estimator, and the "subsample" parameters, which allows one to use a < 1.0 fraction of samples and introduce some regularization.


### We can run the optimization process for this algorithm on a similar grid to the one used for AdaBoost.

In [ ]:
# %%time
# This took about 4.5 minutes on my machine

parameters = {'max_depth':[6,10,None], 'loss':['squared_error','absolute_error'], 
              'n_estimators':[20,50,100], 'learning_rate': [0.1,0.3,0.5]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(GradientBoostingRegressor(), parameters, 
                     cv = KFold(n_splits=5, shuffle=True, random_state = 5), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best params, best score: 0.7370 {'learning_rate': 0.1, 'loss': 'absolute_error', 'max_depth': None, 'n_estimators': 100}


These are comparable to what we obtained with AdaBoost (slightly lower, typically). We can check what happens to the outlier fraction and NMAD.

In [ ]:
bm = model.best_estimator_

In [ ]:
y_pred_bm = cross_val_predict(bm, sel_features, sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle = True, random_state=10))

In [ ]:
print(np.round(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()),3))

print(np.round(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())),3))

0.059
0.037


Overall, the performance of the two algorithms is similar, but one important difference is *timing*. To explore exactly the same parameter space, GBR took ~ 3 times longer than AdaBoost. Additionally, gradient boosted methods typically require more estimators, and we should explore more regularization parameters (e.g. subsampling) as well. In a nutshell, it would be great to speed up things.

### How can we make things faster?

We can improve on the time constraints in two ways: by switching to the histogram-based version of Gradient Boosting Regressor, and by using a Random Search instead of a Grid Search.

HistGradientBoostingRegressor (inspired by [LightGBM](https://lightgbm.readthedocs.io/en/latest/)) works by binning the features into integer-valued bins (the default value is 256, but this parameter can be adjusted; note however that 256 is the maximum!), which greatly reduces the number of splitting points to consider, and results in a vast reduction of computation time, especially for large data sets.

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
# %%time
# This took ~ 18s

parameters = {'max_depth':[6,10,None], 'loss':['squared_error','absolute_error'], 
              'max_iter':[20,50,100], 'learning_rate': [0.1,0.3,0.5]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(HistGradientBoostingRegressor(), parameters, 
                     cv = KFold(n_splits=5, shuffle=True, random_state = 5), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best params, best score: 0.7214 {'learning_rate': 0.3, 'loss': 'squared_error', 'max_depth': None, 'max_iter': 100}


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

,params,mean_test_score,std_test_score,mean_train_score
26,"{'learning_rate': 0.3, 'loss': 'squared_error', 'max_depth': None, 'max_iter': 100}",0.721386,0.036734,0.934003
20,"{'learning_rate': 0.3, 'loss': 'squared_error', 'max_depth': 6, 'max_iter': 100}",0.719629,0.031527,0.889775
8,"{'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': None, 'max_iter': 100}",0.718633,0.039798,0.858452
23,"{'learning_rate': 0.3, 'loss': 'squared_error', 'max_depth': 10, 'max_iter': 100}",0.717640,0.035547,0.923584
5,"{'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 10, 'max_iter': 100}",0.715118,0.039476,0.843148


Even for this relatively small data set, this is much faster (about 15x faster than GradientBoostingRegressor), giving us a chance to explore a wider parameter space (e.g. more trees, more options for learning rate). The trade-off is that we obtain a slight decrease in performance, compared with GBR. However, the standard deviation of test scores over the 5 CV folds suggests that this difference is not statistically significant.

Let's explore a wider parameter space here:

In [ ]:
#%%time
# This took 2 min 42 secs

parameters = {'max_depth':[6,10,None], 'loss':['squared_error','absolute_error'], 
              'max_iter':[100,200,500], 'learning_rate': [0.05, 0.1,0.3,0.5], 
              'early_stopping':[True, False]}
nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(HistGradientBoostingRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best params, best score: 0.7356 {'early_stopping': False, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 10, 'max_iter': 500}


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

,params,mean_test_score,std_test_score,mean_train_score
95,"{'early_stopping': False, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': 10, 'max_i...",0.735611,0.024161,0.950685
98,"{'early_stopping': False, 'learning_rate': 0.1, 'loss': 'squared_error', 'max_depth': None, 'max...",0.734420,0.025424,0.960510
80,"{'early_stopping': False, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': None, 'ma...",0.731776,0.024764,0.923692
115,"{'early_stopping': False, 'learning_rate': 0.3, 'loss': 'squared_error', 'max_depth': None, 'max...",0.727963,0.029271,0.968498
77,"{'early_stopping': False, 'learning_rate': 0.05, 'loss': 'squared_error', 'max_depth': 10, 'max_...",0.726318,0.023250,0.909264


HistGBR seems to improve (as expected) by using more trees and a smaller learning rate.

### Comparison with Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

Finally, we can compare the performance and timings of the grid search above with the option of using a Randomized Search instead. We note that Random Search is usually preferable when we have a high-dimensional parameter space; its use is not particularly warranted here.

The number of iterations (the number of models that are considered) also needs to be adjusted, and depends on the dimensionality of the parameter space as well as the functional dependence of the loss function on the parameters. We will compare the timings with the cell above, where we explore 144 models, and only use 30 for the random search.

The references here explores various ways of running a parameter search.

Bergstra, J. and Bengio, Y., Random search for hyper-parameter optimization, The Journal of Machine Learning Research (2012)

Bergstra, James, et al. "Algorithms for hyper-parameter optimization." Advances in neural information processing systems 24 (2011).

In [ ]:
#%%time
# 31 seconds

parameters = {'max_depth':[6,10,None], 'loss':['squared_error','absolute_error'], 
              'max_iter':[100,200,500], 'learning_rate': [0.05, 0.1,0.3,0.5], 
             'early_stopping':[True, False]}
nmodels = np.product([len(el) for el in parameters.values()])
model = RandomizedSearchCV(HistGradientBoostingRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True, n_iter=30)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best params, best score: 0.7128 {'max_iter': 500, 'max_depth': None, 'loss': 'absolute_error', 'learning_rate': 0.1, 'early_stopping': True}


The Randomized Search was able to find a comparably good solution in less than 1/5 of the time. As we mentioned, the true gains of a Randomized Search pertain to exploring high-dimensional spaces. It is also possible to use the Randomized Search to find the general area of optimal parameters, and then refine the search in that neighborhood with a finer Grid Search.

### Finally, we can compare with XGBoost.

[XGBoost](https://xgboost.readthedocs.io/en/latest/index.html#) stands for “Extreme Gradient Boosting”. It is sometimes known as "regularized" GBM, as it has a default regularization term on the weights of the ensemble, and is more robust to overfitting. It has more flexibility in defining weak learners, as well as the objective (loss) function (note that this doesn't apply to the base estimators, e.g. how splits in trees are chosen, but on the loss that is used to compute pseudoresiduals and gradients). 


In [ ]:
import xgboost as xgb

Medium article explaining XGBoost: [here](https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7); some nice tutorials from XGBoost's site: [here](https://xgboost.readthedocs.io/en/latest/tutorials/index.html)

We can begin by using Grid Search and the original parameter space, in order to compare timings with GBM and HistGBM.

In [ ]:
# %%time

# This took ~ 3 minutes

parameters = {'max_depth':[6,10,None], 'n_estimators':[50,100,200], 
              'learning_rate': [0.1, 0.3,0.5], 'objective':['reg:squarederror','reg:squaredlogerror']}


nmodels = np.product([len(el) for el in parameters.values()])
model = GridSearchCV(xgb.XGBRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

XGBoost is slightly more efficient than GBM, and achieves comparable results on a similar grid. We can use the Random Search to explore some more intensive models (more trees, lower learning rate), and add subsampling as an extra form of regularization.

In [ ]:
%%time

# 3 min 36 secs

parameters = {'max_depth':[6,10,None], 'n_estimators':[50, 100, 200,500], 
              'learning_rate': [0.02,0.05,0.1,0.3], 'objective':['reg:squarederror',
                'reg:squaredlogerror'], 'subsample':[0.5,1]}

nmodels = np.product([len(el) for el in parameters.values()])
model = RandomizedSearchCV(xgb.XGBRegressor(), parameters, cv = KFold(n_splits=5, shuffle=True), \
                     verbose = 2, n_jobs = 4, return_train_score=True, n_iter = 30)
model.fit(sel_features,sel_target.values.ravel())

print('Best params, best score:', "{:.4f}".format(model.best_score_), \
      model.best_params_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best params, best score: 0.7424 {'subsample': 0.5, 'objective': 'reg:squarederror', 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.02}
CPU times: user 18.1 s, sys: 2.8 s, total: 20.9 s
Wall time: 2min 15s


In [ ]:
scores = pd.DataFrame(model.cv_results_)
scoresCV = scores[['params','mean_test_score','std_test_score','mean_train_score']].sort_values(by = 'mean_test_score', \
                                                    ascending = False)
scoresCV.head()

,params,mean_test_score,std_test_score,mean_train_score
10,"{'subsample': 0.5, 'objective': 'reg:squaredlogerror', 'n_estimators': 500, 'max_depth': 10, 'le...",0.750818,0.073358,0.952617
16,"{'subsample': 0.5, 'objective': 'reg:squaredlogerror', 'n_estimators': 200, 'max_depth': 10, 'le...",0.750432,0.074676,0.940629
20,"{'subsample': 1, 'objective': 'reg:squaredlogerror', 'n_estimators': 50, 'max_depth': 10, 'learn...",0.733495,0.074933,0.920086
6,"{'subsample': 1, 'objective': 'reg:squaredlogerror', 'n_estimators': 100, 'max_depth': 10, 'lear...",0.733070,0.068503,0.972264
22,"{'subsample': 1, 'objective': 'reg:squaredlogerror', 'n_estimators': 50, 'max_depth': 10, 'learn...",0.732854,0.068384,0.951915


We are able to get slightly higher scores using this wider parameter space in combination with the Randomized Search, but again, the statistical significance of this increase is very low.

We can also look for outlier fraction and NMAD:

In [ ]:
y_pred_bm = cross_val_predict(bm, sel_features, sel_target.values.ravel(), cv = KFold(n_splits=5, shuffle = True, random_state=5))

In [ ]:
print(len(np.where(np.abs(sel_target.values.ravel()-y_pred_bm)>0.15*(1+sel_target.values.ravel()))[0])/len(sel_target.values.ravel()))

print(1.48*np.median(np.abs(sel_target.values.ravel()-y_pred_bm)/(1 + sel_target.values.ravel())))

0.056128111622007294
0.03729988572517498


### Conclusion: all boosting algorithms behave fairly similarly for this data set. It might be worth simply using the fastest one (HistGBR + Random Search).